# A Guide for Phi-4-mini-instruct on Hopsworks

For details about this Large Language Model (LLM) visit the model page in the HuggingFace repository ➡️ [link](https://huggingface.co/microsoft/Phi-4-mini-instruct)

### 1️⃣ Download Phi-4-mini-instruct using the huggingface_hub library

First, we download the Phi-4 model files (e.g., weights, configuration files) directly from the HuggingFace repository.


In [ ]:
!pip install huggingface_hub --quiet

In [ ]:
# Place your HuggingFace token in the HF_TOKEN environment variable

import os
os.environ["HF_TOKEN"] = "<INSERT_YOUR_HF_TOKEN>"

In [ ]:
import os
from huggingface_hub import snapshot_download

os.environ["HF_HUB_DISABLE_XET"] = "1"

model_id = "microsoft/Phi-4-mini-instruct"
phi_local_dir = snapshot_download(model_id, ignore_patterns="original/*", local_dir=model_id)

## 2️⃣ Register Phi-4-mini-instruct into Hopsworks Model Registry

Once the model files are downloaded from the HuggingFace repository, we can register the models files into the Hopsworks Model Registry.

In [ ]:
import hopsworks

project = hopsworks.login()
mr = project.get_model_registry()

In [ ]:
# The following instantiates a Hopsworks LLM model, not yet saved in the Model Registry

phi = mr.llm.create_model(
    name="phi4_mini_instruct",
    description="Phi-4-mini-instruct model (via HF)"
)

In [ ]:
# Register the Phi model pointing to the local model files

phi.save(phi_local_dir)

## 3️⃣ Deploy Phi-4-mini-instruct

After registering the LLM model into the Model Registry, we can create a deployment that serves it using the vLLM engine with a user-provided configuration (yaml) file.

In [ ]:
# Get a reference to the Phi model if not obtained yet

phi = mr.get_model("phi4_mini_instruct")

In [ ]:
# Upload vllm engine config file for the deployments

ds_api = project.get_dataset_api()

path_to_config_file = f"/Projects/{project.name}/" + ds_api.upload("phi_vllmconfig.yaml", "Resources", overwrite=True)

In [ ]:
phi_depl = phi.deploy(
    name="phi4miniv1",
    description="Phi-4-mini-instruct from HuggingFace",
    config_file=path_to_config_file,
    resources={"num_instances": 1, "requests": {"cores": 1, "memory": 1024*12, "gpus": 1}, "limits": {"cores": 2, "memory": 1024*16, "gpus": 1}},
)

---

In [ ]:
# Retrieve one of the deployments created above

ms = project.get_model_serving()
phi_depl = ms.get_deployment("phi4miniv1")

In [ ]:
phi_depl.start(await_running=60*15) # wait for 15 minutes maximum

In [ ]:
# phi_depl.stop()

In [ ]:
phi_depl.get_state()

## 4️⃣ Prompting Phi-4-mini-instruct

Once the Phi deployment is up and running, we can start sending user prompts to the LLM. You can either use an OpenAI API-compatible client (e.g., openai library) or any other http client.

In [ ]:
import os

openai_v1_uri = phi_depl.get_openai_url()
completions_url = openai_v1_uri + "/completions" 
chat_completions_url = openai_v1_uri + "/chat/completions"

# Resolve API key for request authentication
if "SERVING_API_KEY" in os.environ:
    # if running inside Hopsworks
    api_key_value = os.environ["SERVING_API_KEY"]
else:
    # Create an API KEY using the Hopsworks UI and place the value below
    api_key_value = "<API_KEY>"
    
# Prepare request headers
headers = {
    'Content-Type': 'application/json',
    'Authorization': 'ApiKey ' + api_key_value,
}

### 🟨 Using httpx

In [ ]:
import httpx

In [ ]:
#
# Chat Completion for a user message
#

user_message = "Who is the best French painter. Answer with detailed explanations."

completion_request = {
    "model": phi_depl.name,
    "messages": [
        {
            "role": "user",
            "content": user_message
        }
    ]
}

print("Completion request: ", completion_request, end="\n")

response = httpx.post(chat_completions_url, headers=headers, json=completion_request, timeout=45.0)
print(response)
print(response.json()["choices"][0]["message"]["content"])

In [ ]:
#
# Chat Completion for list of messages
#

messages = [{
    "role": "user",
    "content": "Hi! How are you doing today?"
}, {
    "role": "assistant",
    "content": "I'm doing well! How can I help you?",
}, {
    "role": "user",
     "content": "Can you tell me what the temperate will be in Dallas, in fahrenheit?"
}]


completion_request = {
    "model": phi_depl.name,
    "messages": messages
}

print("Completion request: ", completion_request, end="\n")

response = httpx.post(chat_completions_url, headers=headers, json=completion_request, timeout=45.0)

print(response.json()["choices"][0]["message"]["content"])

### 🟨 Using OpenAI client

In [ ]:
!pip install openai --quiet

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI(
    base_url=openai_v1_uri,
    api_key="X",
    default_headers=headers
)

In [ ]:
#
# Chat Completion for a user message
#

chat_response = client.chat.completions.create(
    model=phi_depl.name,
    messages=[
        {"role": "user", "content": "Who is the best French painter. Answer with a short explanations."},
    ]
)

print(chat_response.choices[0].message.content)

In [ ]:
#
# Chat Completion for list of messages
#

chat_response = client.chat.completions.create(
    model=phi_depl.name,
    messages=[{
        "role": "user",
        "content": "Hi! How are you doing today?"
    }, {
        "role": "assistant",
        "content": "I'm doing well! How can I help you?",
    }, {
        "role": "user",
         "content": "Can you tell me what the temperate will be in Dallas, in fahrenheit?"
    }]
)

print(chat_response.choices[0].message.content)